## Prepare data

In [44]:
import torch
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
import numpy as np
training_device

device(type='cuda')

In [ ]:
# pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" evaluate==0.4 dataset==4.5.0 --upgrade

In [45]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

median_map = {
    "A1": 2,
    "A2": 5,
    "B1": 8,
    "B2": 11,
    "C1": 14,
    "C2": 16
}

df = pd.read_csv("data_v2/sample_data_acc_gec.csv")

#df['ef_level'] = df.apply(lambda row: median_map[row['cefr_level']] if pd.isna(row['ef_level']) else row['ef_level'], axis=1)

df.head()

,recordId,gpt4o_judge_score,gpt4o_evaluation,nova_judge_score,nova_evaluation,llama3_judge_score,llama3_evaluation,majority_value,agreement_percentage,writing_id,task_id,ef_level,activity_instructions,student_submission,strength,area_for_improvement,comment,raw_correction,correction
0,CALL0003862,NaN,NaN,0.0,The student's submission consists of a list of...,5.0,The student demonstrates a high level of contr...,0.0,50.000000,1a7a5582-06e1-4cb5-8374-485b3931288e,111aad8d-1388-499f-b9b9-96b11e2d1c8f,10.0,Read the email from your manager. Then respond...,1.Analyze current workflows to identify ineffi...,[],[],NaN,NaN,NaN
1,CALL0004705,2.0,The student demonstrates basic control over em...,2.0,The student demonstrates a fair level of gramm...,1.0,The student has several grammatical errors: ve...,2.0,66.666667,db061cf3-0ddd-454b-9691-d813a12929e9,fdbea680-2895-4e87-a252-28ff562e6903,8.0,Read the list of issues that you need to discu...,I am really sorry to disturb you but we have s...,"['You started with a polite apology.', 'Your e...","[""Check spelling for words like 'issuses', 'po...",Great job on a clear and well-structured email...,"I am really sorry to disturb you, but we have ...",<input>I am really sorry to disturb you<code a...
2,CALL0007813,3.0,The submission displays control over a variety...,2.0,The student demonstrates a good control of gra...,3.0,The student demonstrates control over complex ...,3.0,66.666667,c61054d2-ebd9-43b7-bfe3-f95f089fd88f,01e27f6c-3c08-4d7c-8c58-fbe93505be01,16.0,Write a paragraph describing how robots are be...,Currently the robots are being used for very p...,['You answered the prompt about robots being u...,['Try adding a brief conclusion to summarize t...,Great job explaining the use and impact of rob...,"Currently, robots are being used for very peti...","<input>Currently<code a=""ORTH"" c=""""> </code><c..."
3,CALL0007674,5.0,The student demonstrates advanced grammatical ...,4.0,The student's submission demonstrates advanced...,4.0,The student demonstrates advanced control over...,4.0,66.666667,596e01cf-addc-45ac-8c94-01d8be746073,51184f66-e317-457a-bda9-d73a241b7871,14.0,"Using your notes, draft a speech with a consis...","Ladies and Gentlemen,\n\nToday, I am thrilled ...",['Clear and engaging speech with motivational ...,['Try adding specific examples to support your...,Great job! Your speech is clear and motivation...,"Ladies and Gentlemen,\n\nToday, I am thrilled ...","<input>Ladies and Gentlemen,\n\nToday, I am th..."
4,CALL0001649,3.0,The student demonstrates good control over bas...,3.0,The student's submission shows a fair control ...,2.0,The student demonstrates good control over gra...,3.0,66.666667,8a1b605f-82ac-4ef0-a1c8-e8c112a05d96,d040caa5-7bd8-412c-aee2-3d9010b91821,12.0,You're going to reply to Alice's blog post abo...,In my culture people are apt to be direct comm...,"[""You clearly explain your own culture's commu...","[""Start with a greeting or reference to Alice'...",Good job explaining your culture! Add a greeti...,"In my culture, people usually communicate dire...","<input>In my culture<code a=""PUNCT"" c="", ""></c..."


In [46]:
df["task_id"].value_counts()

task_id
cfc369d9-74da-4013-be4c-b8148993f658    84
64058d7e-9b06-4ed8-9b01-5b7efb9fb3ef    82
78f976f6-a505-4597-877b-94d9bca3b217    79
d8e67c00-1b68-450a-93f8-ed8f40a1c167    78
0abfaced-7d7a-4d10-b8d6-a1847e6c3913    77
                                        ..
a1c194e8-e83e-4645-90f4-3b99a7b91145    10
a1471208-6755-4814-aa1a-ab276be4d644     9
d2196935-4153-4f5c-a1d2-dae31174ae55     7
e7943e9e-0f41-4fe5-bd2c-906f06a219b1     6
bfc49596-9ed7-4db3-90e3-2e3fa078c99b     4
Name: count, Length: 120, dtype: int64

In [47]:
df['text'] = (
    "Prompt Level: " + df['ef_level'].astype(str) +
    " [SEP] Prompt: " + df['activity_instructions'] +
    " [SEP] Response: " + df['student_submission']
    #+ " [SEP] Correction: " + df['correction']
)

In [48]:
df["text"][1]

"Prompt Level: 8.0 [SEP] Prompt: Read the list of issues that you need to discuss with your boss. Then write only the body of an email to your boss. You do not need to write the addresses, greetings or the closing. Use a formal tone. Type in the input box. Write 80-100 words. Use your own words where possible. [SEP] Response: I am really sorry to disturb you but we have some issuses to revsolve and I'd like to find a solution for everyone in the shortest time possibile. \nFirst of all, I war wondering if I have to go to the conference in Paris next month because flights to France are ending.\nFurthermore I would need to know if we have to postpone the company meeting until after the Parisian conference. I know that it will be next month but we have to reserve the hotel rooms for our guests before they finish.\nIn the end I have to ask a question tu you: who is researching a replacement for you? If no one has been designed, I can help you in the research. "

In [49]:
df = df[["text", "task_id", "ef_level", "majority_value"]]
df = df.rename(columns={'majority_value': 'label'})
df.head()

,text,task_id,ef_level,label
0,Prompt Level: 10.0 [SEP] Prompt: Read the emai...,111aad8d-1388-499f-b9b9-96b11e2d1c8f,10.0,0.0
1,Prompt Level: 8.0 [SEP] Prompt: Read the list ...,fdbea680-2895-4e87-a252-28ff562e6903,8.0,2.0
2,Prompt Level: 16.0 [SEP] Prompt: Write a parag...,01e27f6c-3c08-4d7c-8c58-fbe93505be01,16.0,3.0
3,Prompt Level: 14.0 [SEP] Prompt: Using your no...,51184f66-e317-457a-bda9-d73a241b7871,14.0,4.0
4,Prompt Level: 12.0 [SEP] Prompt: You're going ...,d040caa5-7bd8-412c-aee2-3d9010b91821,12.0,3.0


In [50]:
# Apply the mapping to the 'labels' column
#df['label'] = df['label'].map(label_mapping)
df.dropna(subset=['label', 'text'], inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,text,task_id,ef_level,label
0,Prompt Level: 10.0 [SEP] Prompt: Read the emai...,111aad8d-1388-499f-b9b9-96b11e2d1c8f,10.0,0.0
1,Prompt Level: 8.0 [SEP] Prompt: Read the list ...,fdbea680-2895-4e87-a252-28ff562e6903,8.0,2.0
2,Prompt Level: 16.0 [SEP] Prompt: Write a parag...,01e27f6c-3c08-4d7c-8c58-fbe93505be01,16.0,3.0
3,Prompt Level: 14.0 [SEP] Prompt: Using your no...,51184f66-e317-457a-bda9-d73a241b7871,14.0,4.0
4,Prompt Level: 12.0 [SEP] Prompt: You're going ...,d040caa5-7bd8-412c-aee2-3d9010b91821,12.0,3.0


In [51]:
df["label"].value_counts()

label
2.0    1585
4.0    1270
3.0    1236
5.0    1179
1.0     575
0.0     136
Name: count, dtype: int64

In [52]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['text', 'task_id', 'ef_level', 'label'],
    num_rows: 5981
})

In [53]:
from datasets import ClassLabel, Value, Sequence
new_features = ds.features.copy()
new_features["label"] = ClassLabel(names=[0, 1, 2, 3, 4, 5])
ds = ds.cast(new_features)

# Step 1: Initial train/test split with stratification
train_test_ds = ds.train_test_split(test_size=0.20, seed=20)

# Step 2: Split the test set into half test, half validation
test_valid_split = train_test_ds['test'].train_test_split(test_size=0.5, seed=20)

# Step 3: Combine everything into a single DatasetDict
dataset = DatasetDict({
    'train': train_test_ds['train'],
    'test': test_valid_split['train'],    # This becomes the test set
    'validation': test_valid_split['test']  # This becomes the validation set
})
ds

Casting the dataset:   0%|          | 0/5981 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'task_id', 'ef_level', 'label'],
    num_rows: 5981
})

In [54]:
# Verify label distribution
from collections import Counter

print("Train label counts:", Counter(ds['train']['label']))
print("Test label counts:", Counter(ds['test']['label']))
print("Validation label counts:", Counter(ds['validation']['label']))

KeyError: "Column train not in the dataset. Current columns in the dataset: ['text', 'task_id', 'ef_level', 'label']"

In [ ]:
import json

# Fonction utilitaire pour sauvegarder un split en JSONL
def save_split_to_jsonl(dataset_split, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for record in dataset_split:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

# Sauvegarde des trois splits
save_split_to_jsonl(ds['train'], 'data/train_gec.jsonl')
save_split_to_jsonl(ds['test'], 'data/test_gec.jsonl')
save_split_to_jsonl(ds['validation'], 'data/validation_gec.jsonl')

## Roberta SFT

In [55]:
import numpy as np 
import evaluate

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, cohen_kappa_score, classification_report
from scipy.stats import pearsonr

metric = evaluate.load("accuracy")

In [56]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convertir les logits en classes prédictes

    # 🎯 Exactitude (Accuracy)
    accuracy = accuracy_score(labels, predictions)

    # 🎯 Précision, Rappel et F1-score (pondérés)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    # 🎯 Score de Cohen's Kappa (pondéré)
    cohen_kappa = cohen_kappa_score(labels, predictions, weights="quadratic")

    # 🎯 Corrélation de Pearson
    pearson_corr, _ = pearsonr(labels, predictions)  # Retourne (coef, p-valeur), on garde seulement coef

     # 🎯 Classification Report
    class_report = classification_report(labels, predictions, output_dict=True)  # Get a dictionary of the report


    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "cohen_kappa": cohen_kappa,
        "pearson_corr": pearson_corr,
        "classification_report": class_report  # Add classification report to the return
    }

In [ ]:
from datasets import load_dataset
from datasets import DatasetDict, Dataset

# Charger les fichiers JSONL en DatasetDict
dataset = DatasetDict({
    "train": load_dataset("json", data_files="data/train_gec.jsonl")["train"],
    "test": load_dataset("json", data_files="data/test_gec.jsonl")["train"],
    "valid": load_dataset("json", data_files="data/validation_gec.jsonl")["train"]
})

print(dataset)

In [57]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /home/ec2-user/.cache/huggingface/hub/models--FacebookAI--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/config.json
Model config RobertaConfig {
  "_name_or_path": "FacebookAI/roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_

In [58]:
dataset["train"][0]

{'text': "Prompt Level: 15.0 [SEP] Prompt: You're going to write about an interesting, fictional event that happened to a friend. Write your blog post. Type in the input box. Write 150-200 words. [SEP] Response: My friend is a Star \n\n\nMy freind and me like to know new places for to take a drink, hear good music and seethe crowd.  Was the saturday night and go to for a new pub. But the queue it was huge.  we had been ther fo more onde hour when the security of Pub came to meet with us requesting that into to the pub by end of doors. In that moment i found very strange, but follow in direcion the security. He open the door  and request that look for other security, that which I would be taking to the table where we would be watching the show from the box. He asked if we wanted a drink or something to eat and said it was all on the house. Me and my friend only accepted and smile one for other. When the waiter came until us and left the drinks, called my friend  the Silvester Stalone. A

In [59]:
tok_test = tokenizer(dataset["train"][1]["text"], max_length=256, truncation=True)
tok_test

{'input_ids': [0, 35396, 3320, 12183, 35, 290, 4, 288, 646, 3388, 510, 742, 42944, 35, 1163, 5, 889, 9, 743, 14, 47, 240, 7, 2268, 19, 110, 3504, 4, 1892, 3116, 129, 5, 809, 9, 41, 1047, 7, 110, 3504, 4, 370, 109, 45, 240, 7, 3116, 5, 8480, 6, 17395, 1033, 50, 5, 3172, 4, 7627, 10, 4828, 6328, 4, 7773, 11, 5, 8135, 2233, 4, 21062, 1812, 12, 1866, 1617, 4, 7627, 110, 308, 1617, 147, 678, 4, 646, 3388, 510, 742, 19121, 35, 305, 6998, 24, 28, 678, 7, 278, 10, 529, 19, 47, 116, 50118, 100, 33, 130, 1142, 8, 38, 240, 7, 2268, 59, 141, 7, 432, 19, 106, 4, 50118, 50118, 47326, 13755, 47, 164, 7, 2725, 5, 1019, 220, 353, 116, 50118, 47326, 31231, 52, 342, 160, 5, 138, 529, 454, 71, 5, 1019, 116, 38, 5170, 14, 5, 529, 26012, 268, 40, 45, 2725, 5, 138, 529, 4, 50118, 47326, 12375, 16, 21252, 10, 5010, 13, 69, 116, 38, 5170, 114, 38, 240, 7, 4559, 7404, 9, 5010, 19, 951, 54, 16, 21252, 4, 50140, 50140, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [60]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=312
    )

In [61]:
tokenized_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_test = dataset["test"].map(tokenize_function, batched=True)
tokenized_valid = dataset["validation"].map(tokenize_function, batched=True)

Map:   0%|          | 0/4784 [00:00<?, ? examples/s]

Map:   0%|          | 0/598 [00:00<?, ? examples/s]

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

In [62]:
unique_labels = set(dataset['train']['label'])
num_labels = len(unique_labels)
num_labels

6

In [63]:
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-large", num_labels=num_labels)

loading configuration file config.json from cache at /home/ec2-user/.cache/huggingface/hub/models--FacebookAI--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transform

In [64]:
args = TrainingArguments(
    output_dir="../../../model_saved/roberta-base-ft-acc-writing-gec-task-augmented",
    evaluation_strategy="steps",  # Évaluation aux mêmes intervalles que la sauvegarde
    save_strategy="steps",  # Sauvegarde tous les 500 steps
    save_steps=200,
    eval_steps=200,  # ⚠ IMPORTANT : Évaluation aux mêmes steps
    save_total_limit=4,  # Ne garde que 4 checkpoints max
    learning_rate=1e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine", 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,  
    metric_for_best_model="f1",
    logging_steps=100,
    fp16=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [65]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

Using cuda_amp half precision backend
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/trainer.py:593: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [66]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: task_id, text, ef_level. If task_id, text, ef_level are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4784
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1196
  Number of trainable parameters = 355365894
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-

Step,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: task_id, text, ef_level. If task_id, text, ef_level are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 598
  Batch size = 16
***** Running Evaluation *****
  Num examples = 598
  Batch size = 16
Saving model checkpoint to ../../../model_saved/roberta-base-ft-acc-writing-gec-task-augmented/checkpoint-200
Configuration saved in ../../../model_saved/roberta-base-ft-acc-writing-gec-task-augmented/checkpoint-200/config.json
Saving model checkpoint to ../../../model_saved/roberta-base-ft-acc-writing-gec-task-augmented/checkpoint-200
Configuration saved in ../../../model_saved/roberta-base-ft-acc-writing-gec-task-augmented/checkpoint-200/config.json
Model weights saved in ../../../model_saved/roberta-base-ft-acc-writing-gec-task-augmented/checkpoint-

TrainOutput(global_step=1196, training_loss=0.7992473047313882, metrics={'train_runtime': 602.7212, 'train_samples_per_second': 31.749, 'train_steps_per_second': 1.984, 'total_flos': 1.0867398500395008e+16, 'train_loss': 0.7992473047313882, 'epoch': 4.0})

In [67]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: task_id, text, ef_level. If task_id, text, ef_level are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 598
  Batch size = 16


Trainer is attempting to log a value of "{'0': {'precision': 0.5625, 'recall': 0.5294117647058824, 'f1-score': 0.5454545454545454, 'support': 17.0}, '1': {'precision': 0.5581395348837209, 'recall': 0.4528301886792453, 'f1-score': 0.5, 'support': 53.0}, '2': {'precision': 0.6390977443609023, 'recall': 0.5902777777777778, 'f1-score': 0.6137184115523465, 'support': 144.0}, '3': {'precision': 0.5319148936170213, 'recall': 0.6097560975609756, 'f1-score': 0.5681818181818182, 'support': 123.0}, '4': {'precision': 0.5596330275229358, 'recall': 0.49193548387096775, 'f1-score': 0.5236051502145923, 'support': 124.0}, '5': {'precision': 0.717948717948718, 'recall': 0.8175182481751825, 'f1-score': 0.764505119453925, 'support': 137.0}, 'accuracy': 0.6120401337792643, 'macro avg': {'precision': 0.5948723197222164, 'recall': 0.5819549267950053, 'f1-score': 0.5859108408095378, 'support': 598.0}, 'weighted avg': {'precision': 0.6092859067282929, 'recall': 0.6120401337792643, 'f1-score': 0.60819194341877

{'eval_loss': 0.9818981885910034,
 'eval_accuracy': 0.6120401337792643,
 'eval_precision': 0.6092859067282929,
 'eval_recall': 0.6120401337792643,
 'eval_f1': 0.6081919434187725,
 'eval_cohen_kappa': 0.8447930086654261,
 'eval_pearson_corr': 0.8468238074941701,
 'eval_classification_report': {'0': {'precision': 0.5625,
   'recall': 0.5294117647058824,
   'f1-score': 0.5454545454545454,
   'support': 17.0},
  '1': {'precision': 0.5581395348837209,
   'recall': 0.4528301886792453,
   'f1-score': 0.5,
   'support': 53.0},
  '2': {'precision': 0.6390977443609023,
   'recall': 0.5902777777777778,
   'f1-score': 0.6137184115523465,
   'support': 144.0},
  '3': {'precision': 0.5319148936170213,
   'recall': 0.6097560975609756,
   'f1-score': 0.5681818181818182,
   'support': 123.0},
  '4': {'precision': 0.5596330275229358,
   'recall': 0.49193548387096775,
   'f1-score': 0.5236051502145923,
   'support': 124.0},
  '5': {'precision': 0.717948717948718,
   'recall': 0.8175182481751825,
   'f1-s

## Detailed Evaluation

In [ ]:
list_topic = dataset["valid"]["task_id"]
list_t_set = set(list_topic)
unique_t = (list(list_t_set))

list_level = dataset["valid"]["level_title"]
list_l_set = set(list_level)
unique_l = (list(list_l_set))


In [ ]:
list_r = []

# Assuming 'unique_t' is a list of unique item_ids and 'trainer' is already defined
for t in unique_t:  # Iterate over the first item in unique_t
    sub_ds = tokenized_valid.filter(lambda example: example['task_id'] == t)
    # Get predictions using the trainer
    predictions = trainer.predict(sub_ds)
    # Raw output logits (size [batch_size, num_classes])
    outputs = predictions.predictions
    # Convert logits to predicted class labels (taking the argmax across the classes)
    predicted_labels = np.argmax(outputs, axis=-1)
    ref_label = predictions.label_ids
    # Print or save the predicted classes (this will be a numpy array with the predicted class indices)
    ck = round(cohen_kappa_score(predicted_labels, ref_label, weights="quadratic"), 2)  
    pearson_corr, _ = pearsonr(ref_label, predicted_labels)
    accuracy = accuracy_score(ref_label, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(ref_label, predicted_labels, average="weighted")

    r = {
        "task_id": t,
        "level_title": sub_ds["level_title"][0],
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "ck": ck,
        "pearson": pearson_corr,
        "n_samples": len(sub_ds)
    }
    list_r.append(r)

In [ ]:
list_r_level = []

# Assuming 'unique_t' is a list of unique item_ids and 'trainer' is already defined
for l in unique_l:  # Iterate over the first item in unique_t
    sub_ds = tokenized_valid.filter(lambda example: example['level_title'] == l)
    # Get predictions using the trainer
    predictions = trainer.predict(sub_ds)
    # Raw output logits (size [batch_size, num_classes])
    outputs = predictions.predictions
    # Convert logits to predicted class labels (taking the argmax across the classes)
    predicted_labels = np.argmax(outputs, axis=-1)
    ref_label = predictions.label_ids
    # Print or save the predicted classes (this will be a numpy array with the predicted class indices)
    ck = round(cohen_kappa_score(predicted_labels, ref_label, weights="quadratic"), 2)  
    pearson_corr, _ = pearsonr(ref_label, predicted_labels)
    accuracy = accuracy_score(ref_label, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(ref_label, predicted_labels, average="weighted")

    r = {
        "level_title": sub_ds["level_title"][0],
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "ck": ck,
        "pearson": pearson_corr,
        "n_samples": len(sub_ds)
    }
    list_r_level.append(r)


In [ ]:
import pandas as pd
df_eval_results = pd.DataFrame(list_r, columns=["task_id", "level_title", "accuracy", "precision", "recall", "f1", "ck", "pearson", "n_samples"])
df_eval_results.head(n=10)

In [ ]:
df_eval_results.to_csv("result_eval_data_roberta_large_writing_task_acc.csv", index=False)

In [ ]:
import pandas as pd
df_eval_results_level = pd.DataFrame(list_r_level, columns=["level_title", "accuracy", "precision", "recall", "f1", "ck", "pearson", "n_samples"])
df_eval_results_level.head(n=20)

In [ ]:
df_eval_results_level.to_csv("result_eval_data_roberta_large_acc_by_level.csv", index=False)

## Onnx Export

In [ ]:
#!pip install onnxruntime

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoTokenizer
import torch
import os
from onnxruntime.quantization import quantize_dynamic, QuantType

# === CONFIGURATION ===
# Chemin vers ton dossier contenant le .bin et le config.json
#model_dir = "model_saved/roberta-large-ft-acc-writing-task-augmented/checkpoint-1800"
model_dir="/tmp/tmpdxu3_htb"
onnx_model_path = "model_saved/roberta-large-ft-acc-writing-task-1800.onnx"
quantized_model_path = "model_saved/roberta-large-ft-acc-writing-task-1800-quantized.onnx"

# === ÉTAPE 1 : Charger le modèle et tokenizer ===
model = RobertaForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")
model.eval()

# === ÉTAPE 2 : Préparer un input fictif ===
dummy_text = "Texte d'exemple pour conversion ONNX"
inputs = tokenizer(dummy_text, return_tensors="pt", padding="max_length", max_length=256)

# === ÉTAPE 3 : Exporter vers ONNX ===
torch.onnx.export(
    model,
    (inputs["input_ids"], inputs["attention_mask"]),
    onnx_model_path,
    input_names=["input_ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "logits": {0: "batch_size"},
    },
    opset_version=14  # ⬅️ change ici
)

print(f"✅ Modèle exporté en ONNX : {onnx_model_path}")

# === ÉTAPE 4 : Quantization dynamique ===
quantize_dynamic(
    model_input=onnx_model_path,
    model_output=quantized_model_path,
    weight_type=QuantType.QInt8
)

print(f"✅ Modèle quantifié en ONNX : {quantized_model_path}")

In [ ]:
import onnxruntime

onnx_session = onnxruntime.InferenceSession(onnx_model_path)
onnx_session_quant = onnxruntime.InferenceSession(quantized_model_path)

In [ ]:
max_length = 256  # Ajuste selon la taille maximale de ton modèle

# Fonction d'inférence ONNX
def onnx_infer(input_texts, onnx_model):
    inputs = tokenizer(input_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    input_ids = inputs["input_ids"].numpy()
    attention_mask = inputs["attention_mask"].numpy()
    onnx_inputs = {"input_ids": input_ids, "attention_mask": attention_mask}
    onnx_outputs = onnx_model.run(None, onnx_inputs)
    return onnx_outputs[0]

from tqdm import tqdm

def evaluate_with_metrics(dataset, onnx_model, batch_size=16):
    all_logits = []
    all_labels = []

    # tqdm pour afficher la progression sur les batches
    for i in tqdm(range(0, len(dataset), batch_size), desc="Evaluation"):
        batch = dataset[i:i + batch_size]
        texts = batch["text"]
        labels = batch["label"]

        logits = onnx_infer(texts, onnx_model)
        all_logits.extend(logits)
        all_labels.extend(labels)

    all_logits = np.array(all_logits)
    all_labels = np.array(all_labels)

    # 🔥 Appliquer compute_metrics
    metrics = compute_metrics((all_logits, all_labels))
    return metrics

In [ ]:
valid_data = ds["validation"]  # Ou "valid" selon ton dataset
# === Lancer l'évaluation ===
results = evaluate_with_metrics(valid_data, onnx_model=onnx_session)
print("🎯 Evaluation Results ONNX :")
for k, v in results.items():
    if k == "classification_report":
        print("\n📋 Classification Report :")
        for label, metrics in v.items():
            print(f"{label}: {metrics}")
    else:
        print(f"{k}: {v}")

In [ ]:
valid_data = ds["validation"]  # Ou "valid" selon ton dataset
# === Lancer l'évaluation ===
results = evaluate_with_metrics(valid_data, onnx_model=onnx_session_quant)
print("🎯 Evaluation Results ONNX :")
for k, v in results.items():
    if k == "classification_report":
        print("\n📋 Classification Report :")
        for label, metrics in v.items():
            print(f"{label}: {metrics}")
    else:
        print(f"{k}: {v}")

In [ ]:
import boto3

# Initialize the S3 client
s3 = boto3.client(
    "s3"
)

# Define your bucket name and desired path in S3
bucket_name = "sagemaker-studio-oxs6vznjds"

s3_key = "writing_task_models/accuracy/model_1800_quantized_roberta_large.onnx"  # Change path as needed
# Upload the ONNX file
bucket_path = "sagemaker-studio-oxs6vznjds"
quantized_model_path = "model_saved/roberta-large-ft-acc-writing-task-1800-quantized.onnx"

s3.upload_file(quantized_model_path, bucket_path, s3_key)

print(f"✅ ONNX model uploaded to s3://{bucket_name}/{s3_key}")

## Load Onnx from s3

In [ ]:
import boto3
import onnxruntime as ort

# Define S3 bucket and model key
bucket_name = 'sagemaker-studio-oxs6vznjds'
model_key = 'writing_task_models/accuracy/model_1800_quantized_roberta_large.onnx'
local_model_path = '/tmp/roberta-large-ft-acc-writing-task-1800-quant.onnx'  # or wherever you want to save temporarily

# Initialize boto3 S3 client
s3 = boto3.client('s3')

# Download the ONNX model from S3 to local path
s3.download_file(bucket_name, model_key, local_model_path)

# Load the ONNX model using onnxruntime
session = ort.InferenceSession(local_model_path)

print("ONNX model loaded successfully.")

In [ ]:
valid_data = ds["validation"]  # Ou "valid" selon ton dataset

In [ ]:
results = evaluate_with_metrics(valid_data, onnx_model=session)
print("🎯 Evaluation Results ONNX :")
for k, v in results.items():
    if k == "classification_report":
        print("\n📋 Classification Report :")
        for label, metrics in v.items():
            print(f"{label}: {metrics}")
    else:
        print(f"{k}: {v}")
